#### Training a set of models fot different input parameters.

Import library

In [1]:
%%time

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

plt.rcParams['figure.figsize'] = 12.0, 8.0

from pyts.transformation import GADF,GASF
from sklearn.preprocessing import normalize
from root_numpy import tree2array,array2tree,array2root
import ROOT

import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Activation, Reshape, Conv1D , Conv2D, MaxPooling1D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Embedding, LSTM

Welcome to JupyROOT 6.10/02


Using TensorFlow backend.


CPU times: user 4.15 s, sys: 1.61 s, total: 5.76 s
Wall time: 8.37 s


import data: create a generator to fit

In [2]:
%time
training_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2

def reader_pmt(path):
    extra = np.arange(4096, 4480)
    x_file = ROOT.TFile(path)
    x_tree = x_file.Get("tree")
    x_pmtall = tree2array(x_tree, branches="PMTALL")
    x = np.delete(x_pmtall, extra, axis=1)
    return x

def reader_para(path,para1,para2,para3,para4,para5,para6,para7,para8,para9):
    x_file =  ROOT.TFile(path)
    x_tree = x_file.Get("tree")
    x1 = np.expand_dims(tree2array(x_tree, branches=para1 ),axis=1)
    x2 = tree2array(x_tree, branches=para2 )
    x3 = tree2array(x_tree, branches=para3 )
    x4 = tree2array(x_tree, branches=para4 )
    x5 = tree2array(x_tree, branches=para5 )
    x6 = tree2array(x_tree, branches=para6 )
    x7 = tree2array(x_tree, branches=para7 )
    x8 = tree2array(x_tree, branches=para8 )
    x9 = tree2array(x_tree, branches=para9 )
#     x10 = tree2array(x_tree, branches=para10 )
    
    x = np.stack((x1[:,0],x2[:,1],x3[:,0],x3[:,1],x3[:,2],x3[:,3],x4[:,1],x4[:,7],x4[:,13],x4[:,22],x5[:,33],x5[:,36],x6[:,0],x7[:,1],x8,x9),axis=-1)
    return x

def label(q,k):
    x = np.zeros(len(q))
    for i in range(len(q)):
        x[i] = k
    return x

def sep(q,k,z):
    y = label(q,k)
    x1, x2 ,x3 = np.split(q,[int(len(q)*training_ratio),int(len(q)*(training_ratio+validation_ratio))])
    y1, y2 ,y3 = np.split(y,[int(len(q)*training_ratio),int(len(q)*(training_ratio+validation_ratio))])
    if z == 0:
        return x1, y1
    if z == 1:
        return x2, y2
    if z == 2:
        return x3, y3
    
def comb(one,two,three,four,five,six,seven,portion):
    one1,one2 = sep(one,0,portion)
    two1,two2 = sep(two,1,portion)
    three1,three2 = sep(three,2,portion)
    four1,four2 = sep(four,3,portion)
    five1,five2 = sep(five,4,portion)
    six1,six2 = sep(six,5,portion)
    seven1,seven2 = sep(seven,6,portion)
    
    z = np.concatenate((one1,two1,three1,four1,five1,six1,seven1),axis=0)
    y = np.concatenate((one2,two2,three2,four2,five2,six2,seven2),axis=0)
    return z, y

def comb5(one,two,three,four,five,portion):
    one1,one2 = sep(one,0,portion)    
    two1,two2 = sep(two,1,portion)
    three1,three2 = sep(three,2,portion)
    four1,four2 = sep(four,3,portion)
    five1,five2 = sep(five,4,portion)
    
    z = np.concatenate((one1,two1,three1,four1,five1),axis=0)
    y = np.concatenate((one2,two2,three2,four2,five2),axis=0)
    return z, y

import random
from sklearn.preprocessing import normalize

classes = 7
group = 5

def generator(x,waveform):
    while 1:
        alpha = reader_pmt("Alpha.root")
        bad1 = reader_pmt("BAD_DATA_QUALITY_1.root")
        bad2 = reader_pmt("BAD_DATA_QUALITY_2.root")
        piled = reader_pmt("Piled_2.root")
        ls = reader_pmt("LS.root")
        lsbeta = reader_pmt("lsbeta.root")
        ref = reader_pmt("RefPulse009.root")
        
        z1, z2 = comb(piled,ref,alpha,bad1,bad2,ls,lsbeta,x)
        s = np.random.permutation(np.arange(len(z2)))
        z1 = z1[s]
        z2 = z2[s]
        z1_n = normalize(z1, norm="l1")

        Ratio4us_piled = reader_para("Piled_2.root","Ratio4us","Energy","DataQuality","PSDPara","TriggerCounter","CrystalN","Position_wm","Ratio","TimingDiff")
        Ratio4us_alpha = reader_para("Alpha.root","Ratio4us","Energy","DataQuality","PSDPara","TriggerCounter","CrystalN","Position_wm","Ratio","TimingDiff")
        Ratio4us_ls = reader_para("LS.root","Ratio4us","Energy","DataQuality","PSDPara","TriggerCounter","CrystalN","Position_wm","Ratio","TimingDiff")
        Ratio4us_lsbeta = reader_para("lsbeta.root","Ratio4us","Energy","DataQuality","PSDPara","TriggerCounter","CrystalN","Position_wm","Ratio","TimingDiff")
        Ratio4us_ref = reader_para("RefPulse009.root","Ratio4us","Energy","DataQuality","PSDPara","TriggerCounter","CrystalN","Position_wm","Ratio","TimingDiff")
        
        z3, z4 = comb5(Ratio4us_piled,Ratio4us_ref,Ratio4us_alpha,Ratio4us_ls,Ratio4us_lsbeta,x)
        
        s2 = np.random.permutation(np.arange(len(z4)))
        
        z3 = z3[s2] 
        z4 = z4[s2]

        gasf = GASF(image_size=64, overlapping=False, scale='-1')
        
        for i in range(len(z2)):
            if waveform == "waveform":
                p = np.expand_dims(z1[i],axis=0)
                p = gasf.transform(p)
                q = keras.utils.to_categorical(z2[i],classes)
                yield(p,q)
                
            if waveform == "waveform_1d":
                p = np.expand_dims(z1_n[i],axis=0)
                q = keras.utils.to_categorical(z2[i],classes)
                yield(p,q)
        
        for i in range(len(z3)):
            if waveform == "parameters":
                p = np.expand_dims(z3[i],axis=0)
                q = keras.utils.to_categorical(z4[i],group)
                yield(p,q)
        
# test = generator(0,"waveform")
# print(next(test)[0].shape)
test = generator(1,"waveform_1d")
print(next(test)[0].shape)
test = generator(0,"parameters")
# print(next(test).shape)
print(next(test)[0].shape)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.9 µs
(1, 4096)
(1, 16)


define CNN models

In [8]:
def waveform_classifier():
    model = Sequential()
    model.add(Reshape((64,64,1),input_shape=(64,64)))
    model.add(Conv2D(2, kernel_size=1))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Activation("relu"))
    model.add(Conv2D(4, kernel_size=1,strides=2))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(32,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(7,activation="softmax"))
    model.summary()
    
    waveform = Input(shape=(64,64))
    label = model(waveform)
    
    return Model(waveform,label)

def support_classifier():
    model = Sequential()
    model.add(Dense(130,input_shape=(16,)))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Dense(100))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Dense(30))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Dense(5))
    model.add(Activation("softmax"))
    model.summary()
    
    parameters = Input(shape=(16,))
    label = model(parameters)
    return Model(parameters,label)

def waveform_1d_classifier():
    model = Sequential()
    model.add(Reshape((4096,1),input_shape=(4096,)))
    model.add(Conv1D(4, kernel_size=2, strides=4))
#     model.add(Conv1D(8, kernel_size=2, strides=4))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Activation("relu"))
    model.add(Conv1D(8, kernel_size=2, strides=4))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Activation("relu"))
#     model.add(Conv1D(64, kernel_size=2, strides=4))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    model.add(Dense(32))
    model.add(Activation("relu"))
    model.add(Dense(7))
    model.add(Activation("softmax"))
    model.summary()
    
    waveform = Input(shape=(4096,))
    label = model(waveform)
    
    return Model(waveform,label)

In [9]:
waveform = waveform_classifier()
waveform_1d = waveform_1d_classifier()
parameters = support_classifier()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 2)         4         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 2)         0         
_________________________________________________________________
activation_11 (Activation)   (None, 32, 32, 2)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 4)         12        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 4)           0         
_________________________________________________________________
activation_12 (Activation)   (None, 8, 8, 4)           0         
__________

compile model

In [10]:
waveform.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics=['accuracy'])
waveform_1d.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics=['accuracy'])
parameters.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics=['accuracy'])

train model

In [11]:
%time

train_gen = generator(0,"waveform")
val_gen = generator(1,"waveform")
test_gen = generator(2,"waveform")

steps_t = 21527
steps_vt  = 5727

history = waveform.fit_generator(train_gen,steps_per_epoch=steps_t, epochs=5, verbose=1, validation_data=val_gen,validation_steps=steps_vt)



CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs
Epoch 1/5
21527/21527 [==============================] - 215s - loss: 0.1722 - acc: 0.9268 - val_loss: 0.1288 - val_acc: 0.9495
Epoch 2/5
21527/21527 [==============================] - 203s - loss: 0.1399 - acc: 0.9450 - val_loss: 0.1262 - val_acc: 0.9506
Epoch 3/5
21527/21527 [==============================] - 185s - loss: 0.1352 - acc: 0.9476 - val_loss: 0.1297 - val_acc: 0.9506
Epoch 4/5
21527/21527 [==============================] - 202s - loss: 0.1338 - acc: 0.9480 - val_loss: 0.1157 - val_acc: 0.9534
Epoch 5/5
21527/21527 [==============================] - 198s - loss: 0.1333 - acc: 0.9492 - val_loss: 0.1056 - val_acc: 0.9598


In [6]:
%time

train_gen = generator(0,"waveform_1d")
val_gen = generator(1,"waveform_1d")
test_gen = generator(2,"waveform_1d")

steps_t = 21527
steps_vt  = 3076

history = waveform_1d.fit_generator(train_gen,steps_per_epoch=steps_t, epochs=10, verbose=1, validation_data=val_gen,validation_steps=steps_vt)




CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs
Epoch 1/10
21527/21527 [==============================] - 89s - loss: 0.3702 - acc: 0.8571 - val_loss: 0.3677 - val_acc: 0.8571
Epoch 2/10
21527/21527 [==============================] - 131s - loss: 0.3680 - acc: 0.8571 - val_loss: 0.3723 - val_acc: 0.8571
Epoch 3/10
21527/21527 [==============================] - 99s - loss: 0.3692 - acc: 0.8571 - val_loss: 0.3674 - val_acc: 0.8571
Epoch 4/10
21527/21527 [==============================] - 130s - loss: 0.3691 - acc: 0.8571 - val_loss: 0.3667 - val_acc: 0.8571
Epoch 5/10
21527/21527 [==============================] - 130s - loss: 0.3682 - acc: 0.8571 - val_loss: 0.3706 - val_acc: 0.8571
Epoch 6/10
21527/21527 [==============================] - 103s - loss: 0.3687 - acc: 0.8571 - val_loss: 0.3690 - val_acc: 0.8571
Epoch 7/10
21527/21527 [==============================] - 128s - loss: 0.3682 - acc: 0.8571 - val_loss: 0.3660 - val_acc: 0.8571
Epoch 8/10
21527/21527 [===========

2018-02-26 11:56:23.812405: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX


In [7]:
%time

train_gen = generator(0,"parameters")
val_gen = generator(1,"parameters")
test_gen = generator(2,"parameters")

steps_t = 21527
steps_vt  = 3076

history = parameters.fit_generator(train_gen,steps_per_epoch=steps_t, epochs=4, verbose=1, validation_data=val_gen,validation_steps=steps_vt)




CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs
Epoch 1/4
21527/21527 [==============================] - 110s - loss: 5.1759 - acc: 0.6771 - val_loss: 5.0029 - val_acc: 0.6879
Epoch 2/4
21527/21527 [==============================] - 84s - loss: 5.0136 - acc: 0.6872 - val_loss: 5.0134 - val_acc: 0.6873
Epoch 3/4
21527/21527 [==============================] - 85s - loss: 5.0092 - acc: 0.6875 - val_loss: 5.0009 - val_acc: 0.6880
Epoch 4/4
21527/21527 [==============================] - 83s - loss: 5.0044 - acc: 0.6878 - val_loss: 5.0217 - val_acc: 0.6867
